<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2022/blob/main/NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che4.ipynb -O che4.ipynb
!pip install importnb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from importnb import Notebook
with Notebook(): 
    from che4 import Props

import numpy as np

### NRTL Activity Coefficient Model

$ln (\gamma_i) = \dfrac{ \sum\limits_{j=1}^{n}x_j \tau_{ji}G_{ji}}{\sum\limits_{k=1}^{n}x_kG_{ki}} + 
\sum\limits_{j=1}^{n} \dfrac{x_jG_{ij}}{\sum\limits_{k=1}^{n}x_k G_{kj}} \left( \tau_{ij} - 
\dfrac{\sum\limits_{m=1}^{n} x_m \tau_{mj} G_{mj}}{\sum\limits_{k=1}^{n} x_kG_{kj}} \right)$

$\tau_{ji} = A_{ji} + \dfrac{B_{ji}}{T} + C_{ji}ln(T)+D_{ji}T$

$G_{ji}=exp \left( - \alpha_{ji} \tau_{ji} \right)$

$\alpha_{ij}=\alpha_{ji}$

In [3]:
p = Props(['Methanol','Ethanol','Water'])

In [23]:
p.NRTL_B

array([[   0.  , -155.5 ,  -24.49],
       [ 190.06,    0.  ,  -55.17],
       [ 307.17,  670.44,    0.  ]])

In [28]:
def lng_method1(x, T):
    tau = p.NRTL_B/T
    G = np.exp(-p.NRTL_alpha*tau)
    lng = np.zeros_like(x)
    xtauGdivxG = np.zeros_like(x)
    xG = np.zeros_like(x)
    term2 = np.zeros_like(x)
    N = x.size
    for i in range(N):
        xtauG = 0.
        for j in range(N):
            xtauG += x[j]*tau[j,i]*G[j,i]
            xG[i] += x[j]*G[j,i]
        xtauGdivxG[i] = xtauG/xG[i]

    for i in range(N):
        for j in range(N):
            term2[i]+= x[j]*G[i,j]/xG[j]*(tau[i,j] - xtauGdivxG[j])

    return xtauGdivxG + term2

In [30]:
x=np.array([1/3, 1/3, 1/3])
np.exp(lng_method1(x,300.))

array([0.99367274, 1.14457226, 1.61038962])

In [44]:
a= np.arange(25*6*7).reshape(5,5,6,7)
b = np.arange(7*9*6).reshape(7,9,6)

In [46]:
np.einsum('ijkl,lno->ijkno',a,b).shape

(5, 5, 6, 9, 6)